In [2]:
import argparse
import csv
import itertools
import logging
import time

import requests
import yaml
from bs4 import BeautifulSoup

logging.root.setLevel(logging.INFO)

url = "https://yo.globalvoices.org"

In [3]:
def get_page_soup(url: str) -> BeautifulSoup:

    response = requests.get(url)
    page_html = response.text
    page_soup = BeautifulSoup(page_html, "html.parser")

    # TODO: Complete the function to parse and return the page
    return page_soup


page_soup =get_page_soup(url)

print(page_soup)

<!DOCTYPE html>

<html lang="up">
<head>
<meta charset="utf-8"/>
<title>Global Voices ní-Yorùbá · Ìròyìn ọmọ ìlú jákèjádò Ilé-ayé</title>
<meta content="Ohùn Àgbáyé (Global Voices) jẹ́ àjọṣepọ̀ akọbúlọ́ọ̀gù káríayé t'ó ń ṣe ìtúmọ̀, ìkọ̀ròyìn lórí ọmọ ìlú àgbáyé àti ìgbèjà búlọ́ọ̀gù." name="description">
<meta content="NOODP" name="robots"/>
<!--Mobile viewport settings: Use width of screen instead of a larger viewport, start at normal zoom level, allow scaling by user -->
<meta content="width=device-width, initial-scale=1, user-scalable=yes" name="viewport"/>
<link href="https://yo.globalvoices.org/wp-content/themes/gv-theme/gv-theme.css?updated=2021-07-20" media="screen" rel="stylesheet" type="text/css"/>
<link href="https://yo.globalvoices.org/wp-content/themes/gv-theme/gv-theme-print.css?updated=2021-07-20" media="print" rel="stylesheet" type="text/css"/>
<link href="https://yo.globalvoices.org/wp-content/themes/gv-news-child-theme/style.css?updated=2021-07-20" media="screen" rel="s

In [4]:
def get_valid_urls(page_soup: BeautifulSoup):
    """
    Gets all valid urls from a category page

    input:
        :param: url: category_page
    returns:
        - valid_urls: list of all valid article urls on a given category page
    """
    all_urls = page_soup.findAll("a")
    valid_article_urls = []
    for url in all_urls:
        href = url.get("href")

        if (
            href.startswith("https://yo.globalvoices.org/2020") or href.startswith("https://yo.globalvoices.org/2021") or href.startswith("https://yo.globalvoices.org/2019")
            ):
            story_url = href
            valid_article_urls.append(story_url)
    return list(set(valid_article_urls))

In [17]:
valid_urls = get_valid_urls(page_soup)
print (valid_urls)

['https://yo.globalvoices.org/2020/06/09/740/', 'https://yo.globalvoices.org/2020/05/15/735/', 'https://yo.globalvoices.org/2020/09/', 'https://yo.globalvoices.org/2020/08/16/794/', 'https://yo.globalvoices.org/2019/07/', 'https://yo.globalvoices.org/2019/01/', 'https://yo.globalvoices.org/2019/12/', 'https://yo.globalvoices.org/2019/04/', 'https://yo.globalvoices.org/2019/06/', 'https://yo.globalvoices.org/2019/10/', 'https://yo.globalvoices.org/2020/03/', 'https://yo.globalvoices.org/2019/11/', 'https://yo.globalvoices.org/2020/04/08/681/', 'https://yo.globalvoices.org/2019/02/', 'https://yo.globalvoices.org/2020/08/15/767/', 'https://yo.globalvoices.org/2020/08/', 'https://yo.globalvoices.org/2020/05/', 'https://yo.globalvoices.org/2021/03/', 'https://yo.globalvoices.org/2020/04/', 'https://yo.globalvoices.org/2019/09/', 'https://yo.globalvoices.org/2020/08/11/763/', 'https://yo.globalvoices.org/2019/08/', 'https://yo.globalvoices.org/2020/04/03/640/', 'https://yo.globalvoices.org/2

In [18]:
def get_article_data(article_url: str) -> tuple:
   
    page_soup = get_page_soup(article_url)
    headline = page_soup.find(
        "a", attrs={"rel": "bookmark"}
    )
    
    if headline:
        
        headline = headline.text.strip()

    story_text = " "
    story_div = page_soup.find_all(
        "div", attrs={"class": "entry full-post-content"}
    )
    if story_div:
        all_paragraphs = [div.findAll("p", recursive=False) for div in story_div]
        all_paragraphs = list(itertools.chain(*all_paragraphs))
        story_text = story_text.join(str(paragraph) for paragraph in all_paragraphs)
        story_text = BeautifulSoup(story_text, "html.parser").get_text()
    story_text = story_text if not story_text == " " else None

    return (headline, story_text, article_url)

In [19]:
get_article_data (valid_urls[0])

('Àwọn obìnrin ní Nàìjíríà kojú àdó-olóró àgbẹ́sẹ̀léró àgbàwí ọ̀rọ̀ ìṣèlú lórí ẹ̀rọ-ayélujára',
 "Ní Nàìjíríà, àdó-olóró àgbẹ́sẹ̀léró ni gbàgede àgbàwí ọ̀rọ̀ òṣèlú. Àgbàwí àti ọ̀rọ̀ ìṣèlú sábà máa ń fi ti ọ̀rọ̀ ẹ̀sìn àti ẹlẹ́yàmẹyà ṣe. Àwọn alágbàwí tí ó gbajúmọ̀ lórí ẹ̀rọ-alátagbà\xa0— pàápàá jù lọ lórí Twitter — ní láti lè fi ara gba ọ̀pọ̀lọpọ̀\xa0gbas gbos (èdè àdàlù-mọ́-Gẹ̀ẹ́sì Nàìjíríà fún “ìkànṣẹ́”) ní orí ẹ̀rọ-ayárabíàṣá. Àwọn alágbàwí tí ó jẹ́ obìnrin — ní àfikún sí títẹ àwọn ohun ìdánimọ̀ tí ó ń mú ìpalára wá mọ́lẹ̀ — bákan náà ni wọ́n ń kojú ìkọlù tí ó ti ara ìkórìíra tẹ̀gbintẹ̀gbin ẹ̀yà jẹ yọ. Báwo ni àwọn alágbàwí l'óbìnrin ní Nàìjíríà ṣe ń farada ìkorò orí ẹ̀rọ-ayélujára bíi ìsọ̀rọ àlùfànṣá síni, ọ̀rọ̀ ìkórìíra tẹ̀gbintẹ̀gbin àti ìmọ̀ọ́mọ̀ yí-ọ̀rọ̀ wọn dà? Báwo ni wọ́n ṣe ń tẹ̀síwájú nínú iṣẹ́ wọn tàbí fọnrere iṣẹ́ wọn? Àwọn ajìjàngbara orí ẹ̀rọ alátagbà méjì ní Nàìjíríà fi ìrírí àwọn àgbàwí àti ìkórìíra ìwà abo hàn: #BringBackOurGirls, tí Dókítà Oby Ezekwesili jẹ́ aṣíwájú

In [38]:

def scrape(output_file_name: str, no_of_articles: int):
    
    logging.info("Writing articles to file...")

    with open ("yoruba_global_voices.csv", "w", encoding="utf-8") as csv_file:
        headers = ["headline", "paragraphs", "url"]
        writer = csv.DictWriter(csv_file, delimiter="\t", fieldnames=headers)
        writer.writeheader()
        story_num = 0

        for url in valid_urls:
                headline, paragraphs, url = get_article_data(url)
                if paragraphs:
                    writer.writerow({
                        "headline": headline,
                        "paragraphs": paragraphs,
                        "url": url,
                    })
                    story_num += 1
                    logging.info(f"Successfully wrote story number {story_num}")
                
                if story_num == no_of_articles:
                    logging.info(
                        f"Requested total number of articles {no_of_articles} reached"
                    )
                    logging.info(
                        f"Scraping done. A total of {no_of_articles} articles were scraped!"
                    )
                    #return
                if True:
                    time.sleep(10)
    logging.info(
        f"Scraping done. A total of {story_num} articles were scraped!"
    )

In [39]:
scrape("yoruba_global_voices.csv", -1)

INFO:root:Writing articles to file...
INFO:root:Successfully wrote story number 1
INFO:root:Successfully wrote story number 2
INFO:root:Successfully wrote story number 3
INFO:root:Successfully wrote story number 4
INFO:root:Successfully wrote story number 5
INFO:root:Successfully wrote story number 6
INFO:root:Successfully wrote story number 7
INFO:root:Successfully wrote story number 8
INFO:root:Successfully wrote story number 9
INFO:root:Successfully wrote story number 10
INFO:root:Successfully wrote story number 11
INFO:root:Successfully wrote story number 12
INFO:root:Successfully wrote story number 13
INFO:root:Scraping done. A total of 13 articles were scraped!
